In [1]:
using DataFrames
using TensorFlow
using Distributions
using Images

I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcublas.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:116] Couldn't open CUDA library libcudnn.so.5. LD_LIBRARY_PATH: /usr/local/cuda/lib64
I tensorflow/stream_executor/cuda/cuda_dnn.cc:3459] Unable to load cuDNN DSO
I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcufft.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:125] successfully opened CUDA library libcurand.so.8.0 locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcublas.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcudnn.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcufft.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUD

In [2]:
data_set_folder = "/raided/datasets/sketches_png/png/"
list_of_imgs = open(data_set_folder * "filelist.txt")
img_paths = readlines(list_of_imgs)

20000-element Array{String,1}:
 "airplane/1.png\n" 
 "airplane/10.png\n"
 "airplane/11.png\n"
 "airplane/12.png\n"
 "airplane/13.png\n"
 "airplane/14.png\n"
 "airplane/15.png\n"
 "airplane/16.png\n"
 "airplane/17.png\n"
 "airplane/18.png\n"
 "airplane/19.png\n"
 "airplane/2.png\n" 
 "airplane/20.png\n"
 ⋮                  
 "zebra/19989.png\n"
 "zebra/19990.png\n"
 "zebra/19991.png\n"
 "zebra/19992.png\n"
 "zebra/19993.png\n"
 "zebra/19994.png\n"
 "zebra/19995.png\n"
 "zebra/19996.png\n"
 "zebra/19997.png\n"
 "zebra/19998.png\n"
 "zebra/19999.png\n"
 "zebra/20000.png\n"

In [3]:
test_paths = Array(String, 2500)
path_index = 71
count = 1
for i = 1:2500
    test_paths[i] = img_paths[path_index]
    path_index += 1
    count += 1
    if count % 11 == 0
        path_index += 70
        count = 1
    end
end

In [4]:
files_in_folder = readdir(data_set_folder)
labels = filter(x->isdir(data_set_folder * x) == true, files_in_folder)

250-element Array{String,1}:
 "airplane"     
 "alarm clock"  
 "angel"        
 "ant"          
 "apple"        
 "arm"          
 "armchair"     
 "ashtray"      
 "axe"          
 "backpack"     
 "banana"       
 "barn"         
 "baseball bat" 
 ⋮              
 "umbrella"     
 "van"          
 "vase"         
 "violin"       
 "walkie talkie"
 "wheel"        
 "wheelbarrow"  
 "windmill"     
 "wine-bottle"  
 "wineglass"    
 "wrist-watch"  
 "zebra"        

In [5]:
x = TensorFlow.placeholder(Float32)
y_ = TensorFlow.placeholder(Float32)
W = get_variable("weights", [77841, 250], Float32)
b = get_variable("bias", [250], Float32)

TensorFlow.Variable(<Operation 'bias'>,<Operation 'bias/Assign'>)

In [6]:
y = nn.softmax(x*W + b)
cross_entropy = reduce_mean(-reduce_sum(y_ .* log(y), reduction_indices=[2]))
train_step = train.minimize(train.GradientDescentOptimizer(.00001), cross_entropy)

W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.


<Tensor NoOp:1 shape=unknown dtype=?>

In [7]:
saver = train.Saver()
gpu_options = TensorFlow.tensorflow.GPUOptions(allow_growth=true, per_process_gpu_memory_fraction=0.4)
config = TensorFlow.tensorflow.ConfigProto(log_device_placement=false, gpu_options=gpu_options)
# println(config)
sess = Session(config=config)
train.restore(saver, sess, "4percAcc-allCats")


I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:910] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
I tensorflow/core/common_runtime/gpu/gpu_device.cc:885] Found device 0 with properties: 
name: GeForce GTX 1070
major: 6 minor: 1 memoryClockRate (GHz) 1.683
pciBusID 0000:01:00.0
Total memory: 7.92GiB
Free memory: 7.64GiB
I tensorflow/core/common_runtime/gpu/gpu_device.cc:906] DMA: 0 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:916] 0:   Y 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:975] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0)


In [8]:
type DataLoader
    cur_id::Int
    order::Vector{Int}
end

TrainLoader() = DataLoader(1, shuffle(1:17500))
TestLoader() = DataLoader(1, shuffle(1:2500))

TestLoader (generic function with 1 method)

In [9]:
function get_test(loader::DataLoader)
    x = zeros(Float32, 2500, 77841)
    y = zeros(Float32, 2500, 250)
    for i = 1:2500
        path = test_paths[loader.order[loader.cur_id]][1:end-1]
        sketch_name = split(path, "/")[1]
        img = load(data_set_folder * path)
        img = restrict(restrict(img))
        flatten_img = reshape(img, 77841)
        
        x[i, :] = flatten_img
        label = find((x -> x == sketch_name), labels)[1]
        y[i, label] = 1.0
        loader.cur_id += 1
        if loader.cur_id > 2500
            println("========================")
            println(sketch_name)
            break
        end
    end
    x, y
end

get_test (generic function with 1 method)

In [10]:
correct_prediction = indmax(y, 2) .== indmax(y_, 2)
accuracy=reduce_mean(cast(correct_prediction, Float64))
test_loader = TestLoader()
testx, testy = get_test(test_loader)

println(run(sess, indmax(y, 2), Dict(x=>testx, y_=>testy)))
println(run(sess, indmax(y_, 2), Dict(x=>testx, y_=>testy)))
println(run(sess, accuracy, Dict(x=>testx, y_=>testy)))

head
[168,65,247,32,65,37,28,231,57,190,48,179,53,200,169,60,193,224,223,135,212,193,57,48,84,26,108,67,109,168,62,10,187,126,200,230,190,112,84,58,53,3,247,39,209,32,58,224,26,177,6,48,48,213,76,187,128,233,3,193,190,20,26,36,137,126,209,39,249,20,65,181,193,169,45,63,115,173,26,225,28,20,134,96,212,109,115,119,205,36,58,67,74,168,181,173,33,198,9,83,209,5,25,53,182,109,86,97,65,190,140,39,135,146,207,48,11,60,229,200,109,237,156,241,31,169,115,247,141,246,193,58,193,124,39,149,168,193,152,173,229,109,192,200,55,244,32,53,112,6,191,45,10,32,20,214,233,11,58,26,73,137,66,124,201,149,173,211,26,180,28,7,83,193,17,32,172,33,60,139,5,20,60,190,112,212,229,191,97,229,57,54,106,206,224,168,247,181,190,20,181,82,249,83,228,200,53,244,247,249,133,173,149,128,229,20,179,71,48,229,109,37,37,26,84,84,56,208,67,210,66,7,6,249,139,139,180,115,229,115,72,190,120,66,28,240,84,109,32,209,60,141,128,244,109,67,246,229,97,180,39,73,212,190,181,210,180,45,97,57,126,120,181,190,48,57,6,36,207,83,32,48,11

In [11]:
function sketch_input(path, sketch_name)
    x = zeros(Float32, 1, 77841)
    y = zeros(Float32, 1, 250)
    img = load(path)
    img = restrict(restrict(img))
    flatten_img = reshape(img, 77841)
    x[1, :] = flatten_img
    
    label = find((x -> x == sketch_name), labels)[1]
    y[1, label] = 1.0
    
    x, y
end

sketch_input (generic function with 1 method)

In [12]:
sketch_path = "/raided/datasets/sketches_png/png/bee/1371.png"
sketch_label = "bee"

test_img, test_label = sketch_input(sketch_path, sketch_label)
what_it_thinks = labels[run(sess, indmax(y,2), Dict(x=>test_img, y_=>test_label))[1] + 1]
println("Is it an " * what_it_thinks * "? Answer: ", (sketch_label == what_it_thinks), ".")

Is it an duck? Answer: false.
